In [2]:
import os


def find_csv_files(dir_path):
    csv_files = []
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            if file.endswith(".csv") and '2016' in file and 'NOT' not in file:
                csv_files.append(os.path.join(root, file))
    return csv_files


csv_files = find_csv_files('../raw_eventlogs')
csv_files

In [3]:
import pandas as pd


for file_path in csv_files:
    print(file_path)
    df = pd.read_csv(file_path, sep=';', encoding='latin-1')
    print(df.columns)


In [8]:
import pandas as pd
import os

cols_to_drop = ['ComplaintTheme', 'ComplaintSubtheme', 'ComplaintTopic', 'QuestionTheme', 'QuestionSubtheme', 'QuestionTopic', 'tip', 'service_detail', 'page_action_detail']

activity_mapping = {
    'BPI2016_Complaints.csv': 'make_complaint',
    'BPI2016_Questions.csv': 'ask_question',
    'BPI2016_Werkmap_Messages.csv': 'send_message'
}

timestamp_mapping = {
    'BPI2016_Clicks_Logged_In.csv': 'TIMESTAMP',
    'BPI2016_Complaints.csv': 'ContactDate',
    'BPI2016_Questions.csv': 'ContactDate',
    'BPI2016_Werkmap_Messages.csv': 'EventDateTime'
}

def process_file(file_path):
    print(file_path)
    df = pd.read_csv(file_path, sep=';', encoding='latin-1')
    file_name = os.path.basename(file_path)
    
    if file_name in activity_mapping:
        df['Activity'] = activity_mapping[file_name]
    else:
        df['Activity'] = df['URL_FILE']
        
    print(df.columns)
    df['Timestamp'] = pd.to_datetime(df[timestamp_mapping[file_name]], format='ISO8601')
    
    df['Resource'] = df['Office_U'].astype(str) + '_' + df['Office_W'].astype(str)
    
    return df

dfs = []
for file_path in csv_files:
    dfs.append(process_file(file_path))

combined_df = pd.concat(dfs, ignore_index=True)
combined_df = combined_df.sort_values(['CustomerID', 'Timestamp'])
combined_df = combined_df.drop(cols_to_drop, axis=1)

In [9]:
max(combined_df['Timestamp'])